In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


from __future__ import unicode_literals, print_function, division
import pickle as pkl
import os
from io import open
import unicodedata
import string
import re
import random
import torch
import sacrebleu
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import numpy as np
plt.switch_backend('agg')
import matplotlib.ticker as ticker
# get_ipython().run_line_magic('matplotlib', 'inline')
import torch.nn as nn
from torch import optim
from torch.optim.lr_scheduler import ExponentialLR
import torch.nn.functional as F
from torch.autograd import Variable
import time
import math
from sacrebleu import raw_corpus_bleu, corpus_bleu

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device
#print('VI_preidx_RNN_attn_lr1e3_hidden512_TF.py')

# In[25]:


'''old version of data loaders. build the lang without pretrained index'''
# SOS_token = 0
# EOS_token = 1
# PAD_IDX = 2
# UNK_IDX = 3
# class Lang:
#     def __init__(self, name):
#         self.name = name
#         self.word2index = {}
#         self.word2count = {}
#         self.index2word = {0: "SOS", 1: "EOS", 2:"PAD", 3:"UNK"}
#         self.n_words = 4  # Count SOS and EOS

#     def addSentence(self, sentence):
#         for word in sentence:
#             self.addWord(word)

#     def addWord(self, word):
#         if word not in self.word2index:
#             self.word2index[word] = self.n_words
#             self.word2count[word] = 1
#             self.index2word[self.n_words] = word
#             self.n_words += 1
#         else:
#             self.word2count[word] += 1

            
# def loadingLangs(sourcelang, targetlang, setname):
#     input_ls = []
#     output_ls = []
#     print('Reading lines...')
#     # Read the file 
#     with open('/home/jh5695/nlp_project/nlp_data/iwslt-%s-%s/%s.tok.%s'%(sourcelang, targetlang, setname,sourcelang)) as f:
#         for line in f.readlines():
#             input_ls.append([normalizeString(word) for word in line.split()])
#     with open('/home/jh5695/nlp_project/nlp_data/iwslt-%s-%s/%s.tok.%s'%(sourcelang, targetlang, setname,targetlang)) as f:
#         for line in f.readlines():
#             output_ls.append([normalizeString(word) for word in line.split()])
#     pairs = list(zip(input_ls, output_ls))
#     print('Read %s sentence pairs'%(len(input_ls)))
#     input_lang = Lang(sourcelang)
#     output_lang = Lang(targetlang)
#     print("Counting words...")
#     for pair in pairs:
#         input_lang.addSentence(pair[0])
#         output_lang.addSentence(pair[1])
#     print("Counted words:")
#     print(input_lang.name, input_lang.n_words)
#     print(output_lang.name, output_lang.n_words)
#     return input_lang, output_lang, pairs


# In[2]:


data_dir = '/home/jh5695/nlp_project/nlp_data'
SOS_token = 0
EOS_token = 1
PAD_IDX = 2
UNK_IDX = 3
VOCAB_SIZE = 50000


def pkl_loader(file_name):
    with open(file_name+'.p', 'rb') as f:
        objct = pkl.load(f)
    return(objct)

def pkl_dumper(obj, file_name):
    with open(file_name+'.p', 'wb') as f:
        pkl.dump(obj, f, protocol=None)

def load_pretrained_wordvec(lan):
    if lan == 'zh':
        filename = 'wiki.zh.vec'
    if lan == 'en':
        filename = 'wiki-news-300d-1M.vec'
    if lan == 'vi':
        filename = 'wiki.vi.vec'
        
    with open(os.path.join(data_dir, filename),encoding='utf-8') as f:
        word_vecs = np.zeros((VOCAB_SIZE+4, 300))
        word_vecs[UNK_IDX] = np.random.normal(scale=0.6, size=(300, ))
        word_vecs[SOS_token] = np.random.normal(scale=0.6, size=(300, ))
        word_vecs[EOS_token] = np.random.normal(scale=0.6, size=(300, ))

        words_ft = {'<pad>': PAD_IDX,
                   '<unk>': UNK_IDX, 
                   '<sos>': SOS_token,
                   '<eos>': EOS_token}
        idx2words_ft = {PAD_IDX:'<pad>', UNK_IDX: '<unk>', SOS_token: '<sos>', EOS_token: '<eos>'}
        ordered_words_ft = ['<sos>', '<eos>', '<pad>', '<unk>']
        count = 0
        for i, line in enumerate(f):
            if i == 0:
                continue
            if len(idx2words_ft) >= VOCAB_SIZE: 
                break
            s = line.split()
            if (np.asarray(s[1:]).size != 300):
                print(lan, i, np.asarray(s[1:]).size)
                continue
            word_vecs[count+4, :] = np.asarray(s[1:])
            words_ft[s[0]] = count+4
            idx2words_ft[count+4] = s[0]
            ordered_words_ft.append(s[0])
            count += 1
    word_vecs = torch.FloatTensor(word_vecs)
    pkl_dumper(word_vecs, os.path.join(data_dir, lan + '_word_vecs'))
    pkl_dumper(words_ft, os.path.join(data_dir, lan + '_words_ft'))
    pkl_dumper(idx2words_ft, os.path.join(data_dir, lan + '_idx2words_ft'))
    pkl_dumper(ordered_words_ft, os.path.join(data_dir, lan + '_ordered_words_ft'))

In [ ]:

load_pretrained_wordvec('vi')
load_pretrained_wordvec('en')


In [2]:
# In[4]:
word_vecs = {}
word2index = {}
index2word = {}
word_vecs['en'] = pkl_loader('/home/jh5695/nlp_project/nlp_data/en_word_vecs')
word_vecs['vi'] = pkl_loader('/home/jh5695/nlp_project/nlp_data/vi_word_vecs')
word2index['en'] = pkl_loader('/home/jh5695/nlp_project/nlp_data/en_words_ft')
word2index['vi'] = pkl_loader('/home/jh5695/nlp_project/nlp_data/vi_words_ft')
index2word['en'] = pkl_loader('/home/jh5695/nlp_project/nlp_data/en_idx2words_ft')
index2word['vi'] = pkl_loader('/home/jh5695/nlp_project/nlp_data/vi_idx2words_ft')


In [3]:
VOCAB_SIZE = len(word2index['en'])
VOCAB_SIZE 

50000

In [4]:
class Lang:
    def __init__(self, name, index2word, word2index):
        self.name = name
        self.word2index = word2index
        self.index2word = index2word
        self.n_words = len(index2word)



import html
def normalizeString(s):
    s = re.sub(r"([.!?])", r" \1", s)
    s = html.unescape(s)
    return s



def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.5)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.show()


def loadingLangs(sourcelang, targetlang, setname):
    input_ls = []
    output_ls = []
    print('Reading lines...')
    # Read the file 
    with open(data_dir+'/iwslt-%s-%s/%s.tok.%s'%(sourcelang, targetlang, setname,sourcelang), encoding='utf-8') as f:
        for line in f.readlines():
            input_ls.append([normalizeString(word) for word in line.split()])
    with open(data_dir+'/iwslt-%s-%s/%s.tok.%s'%(sourcelang, targetlang, setname,targetlang), encoding='utf-8') as f:
        for line in f.readlines():
            output_ls.append([normalizeString(word) for word in line.split()])
    pairs = list(zip(input_ls, output_ls))
    pairs = [pair for pair in pairs if (len(pair[0])+len(pair[1]))>0]
    print('Read %s sentence pairs'%(len(input_ls)))
    if sourcelang == 'zh':
        input_lang = Lang(sourcelang, index2word['zh'], word2index['zh'])
    if sourcelang =='vi':
        input_lang = Lang(sourcelang, index2word['vi'], word2index['vi'])
    output_lang = Lang(targetlang, index2word['en'], word2index['en'])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [5]:

source_tra, target_tra, pairs_tra = loadingLangs('vi', 'en', 'train')
source_val, target_val, pairs_val = loadingLangs('vi', 'en', 'dev')
source_tes, target_tes, pairs_tes = loadingLangs('vi', 'en', 'test')

Reading lines...
Read 133317 sentence pairs
Counted words:
vi 50000
en 50000
Reading lines...
Read 1268 sentence pairs
Counted words:
vi 50000
en 50000
Reading lines...
Read 1553 sentence pairs
Counted words:
vi 50000
en 50000


In [8]:
source_pre_trained_emb = word_vecs['en']

In [7]:
target_pre_trained_emb = word_vecs['vi']

In [9]:

# In[9]:


# print("90% of vit sentences length = {0}".format(np.percentile([len(x[0]) for x in pairs_tra], 90)))
# print("90% of english sentences length = {0}".format(np.percentile([len(x[1]) for x in pairs_tra], 90)))
# print(random.choice(pairs_tra))


# In[10]:


MAX_SENT_LEN = 38
BATCH_SIZE = 64


In [10]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] if word in lang.word2index else UNK_IDX for word in sentence]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair,source,target):
    input_lang = source
    output_lang = target
    input_tensor = tensorFromSentence(input_lang, pair[0]).reshape((-1))
    target_tensor = tensorFromSentence(output_lang, pair[1]).reshape((-1))
    return (input_tensor, input_tensor.shape[0], target_tensor, target_tensor.shape[0])


# In[12]:


class NMTDataset(Dataset):
    def __init__(self, source, target, pairs):
        self.source = source
        self.target = target
        self.pairs = pairs
        
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        inp_ten, inp_len, tar_ten, tar_len = tensorsFromPair(self.pairs[key], self.source, self.target)
        item = {}
        item['inputtensor'] = inp_ten[:MAX_SENT_LEN]
        item['inputlen'] = min(inp_len, MAX_SENT_LEN)
        item['targettensor'] = tar_ten[:MAX_SENT_LEN]
        item['targetlen'] = min(tar_len, MAX_SENT_LEN)
        return item

In [11]:

train_data = NMTDataset(source_tra, target_tra, pairs_tra)
val_data = NMTDataset(source_tra, target_tra, pairs_val)
test_data = NMTDataset(source_tra, target_tra, pairs_tes)


In [12]:
# ## Dataloader
#collate function

def collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all
    data have the same length
    """
    src_data, tar_data, src_len, tar_len = [], [], [], []
    for datum in batch:        
        src_datum = np.pad(np.array(datum['inputtensor']),
                                pad_width=((0,MAX_SENT_LEN-datum['inputlen'])),
                                mode="constant", constant_values=PAD_IDX)
        tar_datum = np.pad(np.array(datum['targettensor']),
                                pad_width=((0,MAX_SENT_LEN-datum['targetlen'])),
                                mode="constant", constant_values=PAD_IDX)
        src_data.append(src_datum)
        tar_data.append(tar_datum)
        src_len.append(datum['inputlen'])
        tar_len.append(datum['targetlen'])
        
    ind_dec_order = np.argsort(src_len)[::-1]
    src_data = np.array(src_data)[ind_dec_order]
    src_len = np.array(src_len)[ind_dec_order]
    tar_data = np.array(tar_data)[ind_dec_order]
    tar_len = np.array(tar_len)[ind_dec_order]
    return [torch.from_numpy(np.array(src_data)).to(device),torch.from_numpy(np.array(tar_data)).to(device),
                torch.LongTensor(np.array(src_len)).to(device),torch.LongTensor(np.array(tar_len)).to(device)]

In [13]:
train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=BATCH_SIZE,shuffle=True, collate_fn=collate_func)
val_loader = torch.utils.data.DataLoader(val_data,
                                           batch_size=BATCH_SIZE,shuffle=False, collate_fn=collate_func)


In [14]:
hyper = {
    'HIDDEN_SIZE': 512,
    'LR': 0.0001,
    'EVA_EVERY': 100,
    'DROP_OUT': 0,
    'TEACHER_RATIO': 1,
    'N_LAYERS': 1,
    'KER_SIZE': 3,
    'NUM_EPOCHS': 30   
}

In [53]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, pretra = False,n_layers=1, dropout=0.5):
        super(EncoderRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embed_size = embed_size
        self.n_layers = n_layers
        self.dropout = dropout
        if pretra:
            emb_mat = torch.from_numpy(source_pre_trained_emb.numpy()).float()
            self.embedding = nn.Embedding.from_pretrained(emb_mat,freeze = True)
        else:
            self.embedding = nn.Embedding(input_size,embed_size,padding_idx = PAD_IDX)
        self.gru = nn.GRU(embed_size, hidden_size, n_layers, dropout=self.dropout, bidirectional=True)
    
    

    def forward(self, input_seqs, input_lengths, hidden=None):
        '''
        :param input_seqs: 
            Variable of shape (num_step(T),batch_size(B)), sorted decreasingly by lengths(for packing)
        :param input:
            list of sequence length
        :param hidden:
            initial state of GRU
        :returns:
            GRU outputs in shape (T,B,hidden_size(H))
            last hidden stat of RNN(i.e. last output for GRU)
        '''
        embedded = self.embedding(input_seqs)
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        outputs, hidden = self.gru(packed, hidden)
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs)  # unpack (back to padded)
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, :, self.hidden_size:]  # Sum bidirectional outputs
#         print('enc, output size', outputs.size())
#         print('enc, hidden size', hidden.size())
        return outputs, hidden


# In[28]:

#Some code was inspired by Bahdanau's work.
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        self.hidden_size = hidden_size
        self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))
        stdv = 1. / math.sqrt(self.v.size(0))
        self.v.data.normal_(mean=0, std=stdv)

    def forward(self, hidden, encoder_outputs):
        
        max_len = encoder_outputs.size(0)
        Hid_copy = hidden.repeat(max_len,1,1).transpose(0,1)
        encoder_outputs = encoder_outputs.transpose(0,1) 
        attn_energies = self.score(Hid_copy,encoder_outputs) 
        return F.softmax(attn_energies).unsqueeze(1) 

    def score(self, hidden, encoder_outputs):
        score = F.tanh(self.attn(torch.cat([hidden, encoder_outputs], 2)))
        score = score.transpose(2,1) 
        v = self.v.repeat(encoder_outputs.data.shape[0],1).unsqueeze(1) 
        score = torch.bmm(v,score) 
        return score.squeeze(1) 

class AttnDecoderRNN(nn.Module):
    def __init__(self, hid_size, emb_size, out_size, pretra = False,n_layers=1):
        super(AttnDecoderRNN, self).__init__()
       
        self.n_layers = n_layers
        self.hid_size = hid_size
        self.emb_size = emb_size
        self.out_size = out_size
          
        if pretra:
            emb_mat = torch.from_numpy(target_pre_trained_emb.numpy()).float()
            self.embedding = nn.Embedding.from_pretrained(emb_mat,freeze = True)
        else:
            self.embedding = nn.Embedding(out_size, emb_size)
       
        self.attn = Attn('concat', hid_size)
        self.gru = nn.GRU(hid_size + emb_size, hidden_size, n_layers, dropout=0.1)
      
        self.out = nn.Linear(hid_size, out_size)



    def forward(self, word_input, last_hidden, encoder_outputs):
        
        
#         print('dec, word input', word_input.size())
        word_embedded = self.embedding(word_input.squeeze()).view(1, -1, self.emb_size) 
       
#         print('dec, wembd', word_embedded.size())
        
#         print('dec, last hidden', last_hidden.size())
#         print('dec, enc outp', encoder_outputs.size())
        attn_weights = self.attn(last_hidden[-1], encoder_outputs)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))  
        context = context.transpose(0, 1) 
#         print('dec, context', context.size())
#         print('dec, wembd', word_embedded.size())
        
        rnn_input = torch.cat((word_embedded, context), 2)
#         print('dec rnn input', rnn_input.size())
#         print('dec last hidden', last_hidden.size())
        
        output, hidden = self.gru(rnn_input, last_hidden[-1].unsqueeze(0))
        output = output.squeeze(0)  
        
        output = F.log_softmax(self.out(output))
        
        return output, hidden, attn_weights


# In[29]:


def train(input_tensor, target_tensor, input_len, target_len, encoder, decoder,
          encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_SENT_LEN):
    
    hidden_size = hyper['HIDDEN_SIZE']
    learning_rate = hyper['LR']
    dropout_p = hyper['DROP_OUT']
    teacher_forcing_ratio = hyper['TEACHER_RATIO']
    n_layers = hyper['N_LAYERS']
    ker_size = hyper['KER_SIZE']
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    scheduler_encoder = ExponentialLR(encoder_optimizer, 0.95, last_epoch=-1) 
    scheduler_decoder = ExponentialLR(decoder_optimizer, 0.95, last_epoch=-1) 

    max_input_len = max(input_len)
    max_target_len = max(target_len)
    
    batch_size = input_tensor.size()[1]
    
#     encoder_hidden = encoder.initHidden(batch_size)
    
    loss = 0
    
    # feed-forward layer resulting encoder outputs, ei refers to each word token in input sentence
    encoder_outputs, encoder_hidden = encoder(input_tensor, input_len)
    decoder_input = torch.tensor([[SOS_token]*batch_size], device=device) 
    
#     decoder_hidden = nn.Linear(2*hidden_size,hidden_size)(
#             torch.cat((encoder_hidden[0].cpu(),encoder_hidden[1].cpu()),dim = 1)).to(device).unsqueeze(0).to(device)
   # print('encoder_hidden size:', encoder_hidden.size())
    decoder_hidden = encoder_hidden#torch.cat([encoder_hidden[0, :, :].unsqueeze(0), encoder_hidden[1, :, :].unsqueeze(0)], dim = 0)

     
    #print('input to decoder:', decoder_input.size(), decoder_hidden.size(), encoder_outputs.size())
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing: 
        for di in range(max_target_len):
            #if mode_dec == 'attn':
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
                
            #else:
                #decoder_output, decoder_hidden = decoder(
                    #decoder_input, decoder_hidden, encoder_outputs)
            decoder_input = target_tensor[di].view(batch_size,1)  
            loss += criterion(decoder_output, target_tensor[di])    
    else:
  
        for di in range(max_target_len):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach().unsqueeze(1)
#             torch.nn.utils.clip_grad_norm_(encoder.parameters(), max_norm=5)
#             torch.nn.utils.clip_grad_norm_(decoder.parameters(), max_norm=5)
#             print('loss dec', decoder_output.size())
#             print('loss target tensor', target_tensor.size())
    
            loss += criterion(decoder_output, target_tensor[di])

    loss.backward()
    torch.nn.utils.clip_grad_norm_(encoder.parameters(), 3)
    torch.nn.utils.clip_grad_norm_(decoder.parameters(), 3)
    encoder_optimizer.step() 
    decoder_optimizer.step()

    return loss.item() / float(max_target_len)


# In[30]:


def bleu_new(corpus,truths):
    n = len(corpus)
    #bleu = [0]*n
    pred_ls = []
    true_ls = []
    for i in range(n):
        pred, true = corpus[i], truths[i]
        pred_ls.append( [convert_idx_2_sent_new(sent, target_tra) for sent in pred])
        true_ls.append([convert_idx_2_sent_new(sent, target_tra) for sent in true])
    flattened_pred  = [val for sublist in pred_ls for val in sublist]
    flattened_true  = [val for sublist in true_ls for val in sublist]
    bleu= corpus_bleu(flattened_pred, [flattened_true]).score
    return bleu

# In[49]:


def convert_idx_2_sent_new(idx_tensor, lang_obj):
    word_list = []
    #truth_word_list = []
    for i in idx_tensor:
        if i.item() not in set([PAD_IDX,EOS_token,SOS_token]):
            word_list.append(lang_obj.index2word[i.item()])
#     for j in truth_tensor:
#         if j.item() not in set([PAD_IDX,EOS_token,SOS_token]):
#             truth_word_list.append(lang_obj.index2word[j.item()])
    sent = (' ').join(word_list)
    #truth_sent = (' ').join(truth_word_list)
    return sent


# In[31]:


def evaluate(encoder, decoder, data_loader, mode_enc, mode_dec, max_length=MAX_SENT_LEN):
    start = time.time()
    hidden_size = hyper['HIDDEN_SIZE']
    learning_rate = hyper['LR']
    dropout_p = hyper['DROP_OUT']
    teacher_forcing_ratio = hyper['TEACHER_RATIO']
    n_layers = hyper['N_LAYERS']
    ker_size = hyper['KER_SIZE']
    encoder.eval()
    decoder.eval()
    inputs = []
    corpus = []
    truths = []
    for i, (input_sentences, target_sentences,len1,len2) in enumerate(data_loader):
#         if i % 5 == 0:
#             print('Time: {}, Step: [{}/{}]'.format(
#                 timeSince(start, i + 1/len(train_loader)), i, len(data_loader)))
        
        input_tensor = input_sentences.transpose(0,1).to(device) #L*B
        target_tensor = target_sentences.transpose(0,1).to(device)
    
        inputs.append(input_sentences)
        truths.append(target_sentences)
        
        max_input_len = max(len1)
        max_target_len = max(len2)
        
        
        batch_size = input_tensor.size()[1]

        encoder_outputs, encoder_hidden = encoder(input_tensor, len1)

        decoder_input = torch.tensor([[SOS_token]*batch_size], device=device) 
        
        decoder_hidden = encoder_hidden
        
        decoded_words = torch.zeros(batch_size, int(max_input_len*1.5))
    
        for di in range(int(max_input_len*1.5)):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)        
            topv, topi = decoder_output.topk(1) #topi B*1
            decoded_words[:,di] = topi.squeeze()
            decoder_input = topi.squeeze().detach().unsqueeze(1)
            
        corpus.append(decoded_words)
#         print(inputs[0].size(), corpus[0].size(), truths[0].size()) # all B*L
    return inputs, corpus, truths

In [38]:
source_tra.n_words

50000

In [30]:
source_pre_trained_emb.size()

torch.Size([50004, 300])

In [31]:
target_pre_trained_emb.size()

torch.Size([50004, 300])

In [55]:
# In[ ]:


start = time.time()
hidden_size = hyper['HIDDEN_SIZE']
learning_rate = hyper['LR']
eva_every = 1#hyper['EVA_EVERY']
dropout_p = hyper['DROP_OUT']
teacher_forcing_ratio = 1 #hyper['TEACHER_RATIO']
n_layers = hyper['N_LAYERS']
ker_size = hyper['KER_SIZE']
num_epoch = hyper['NUM_EPOCHS']
early_stopping = False
patience = 3
required_progress = 0.01

# input_size, embed_size, hidden_size,
encoder = EncoderRNN(source_tra.n_words, 300, hidden_size,False).to(device)

#     else:
#         encoder = EncoderCNN(source_tra.n_words, hidden_size, word_vecs=word_vecs[source_tra.name], dropout_p=dropout_p, ker_size=ker_size).to(device)

encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)

#     if mode_dec == 'attn':
#             decoder = AttnDecoder(hidden_size, target_tra.n_words, dropout_p=dropout_p).to(device)
#     else:
decoder = AttnDecoderRNN(hidden_size, 300, target_tra.n_words,False, n_layers=1).to(device)

decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.NLLLoss() 
plot_bleu_score_val = []
plot_losses = []
loss_total = 0 
best_score = None
count = 0
filename = 'VI_best_rnnattn_preidx_lr1e3_hidden512_TF' #########
whole_val_bleu = []
whole_train_loss = []
for epoch in range(1, num_epoch + 1): 
    plot_bleu_score_val = []
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0
    for i, (input_sentences, target_sentences,len1,len2) in enumerate(train_loader): 
      ### delete break

        encoder.train()
        decoder.train()
        input_tensor = input_sentences.transpose(0,1).to(device)    
        target_tensor = target_sentences.transpose(0,1).to(device)
        loss = train(input_tensor, target_tensor, len1, len2, encoder,
                   decoder, encoder_optimizer, decoder_optimizer, criterion)
        loss_total += loss
        
        if i > 0 and i % hyper['EVA_EVERY'] == 0:
            inputs, corpus, truths = evaluate(encoder, decoder, val_loader, max_length=MAX_SENT_LEN, mode_enc='rnn', mode_dec='noattn')
#              bleu_score_val = bleu_new(corpus, truths)
            bleu_score_val_avg = bleu_new(corpus, truths)#np.mean(bleu_score_val)
            loss_avg = loss_total / eva_every
            loss_total = 0
            plot_losses.append(loss_avg)
            plot_bleu_score_val.append(bleu_score_val_avg)
            if best_score is None:
                best_score = bleu_score_val_avg
            if bleu_score_val_avg < best_score + required_progress:
                count += 1
            elif bleu_score_val_avg > best_score:
                state = {'epoch': epoch + 1, 
                           'state_dict_enc': encoder.state_dict(),
                           'state_dict_dec': decoder.state_dict(), 
                           'best_accuracy': best_score, 
                           'optimizer_enc': encoder_optimizer.state_dict(),
                          'optimizer_dec': decoder_optimizer.state_dict()}
                print ('new best achieved')
                torch.save(state, filename+'.pth')
                pkl_dumper(inputs, os.path.join(data_dir, 'VI_preidx_attn_lr1e3_hidden512_inputs_TF'))
                pkl_dumper(corpus,os.path.join(data_dir, 'VI_preidx_attn_lr1e3_hidden512_pred_corpus_TF'))
                pkl_dumper(truths, os.path.join(data_dir,'VI_preidx_attn_lr1e3_hidden512_target_truths_TF'))
                #  
                best_score = bleu_score_val_avg
                #count = 0
#             if early_stopping:
#                 if count >= patience:
#                     print("earily stop triggered")
#                     break
            print('-----------------------------------------')
            print('Time: {0}, Epoch: [{1}/{2}], Step: [{3}/{4}], Train Loss: {5}, BLEU score: {6}'.format(
                  timeSince(start, i + 1/len(train_loader)), epoch, num_epoch, i, 
                  len(train_loader), loss_avg, bleu_score_val_avg))
            print('\nInput> %s'%(' '.join([source_tra.index2word[i.item()] for i in inputs[0][3] if i.item() not in set([PAD_IDX,EOS_token,SOS_token])])))
            print('\nTarget= %s'%(convert_idx_2_sent_new(truths[0][3], target_tra)),
              '\nPredict< %s' %(convert_idx_2_sent_new(corpus[0][3], target_tra)))

            print('\nInput2> %s'%(' '.join([source_tra.index2word[i.item()] for i in inputs[0][13] if i.item() not in set([PAD_IDX,EOS_token,SOS_token])])))
            print('\nTarget2= %s'%(convert_idx_2_sent_new(truths[0][13], target_tra)),
              '\nPredict2< %s' %(convert_idx_2_sent_new(corpus[0][13], target_tra)))
    whole_train_loss.append(plot_losses)
    whole_val_bleu.append(plot_bleu_score_val)
#             print('-----------------------------------------')
    pkl_dumper(whole_train_loss, os.path.join(data_dir, 'VI_preidx_attn_train_lr1e3_hidden512_loss_TF'))
    pkl_dumper(whole_val_bleu,os.path.join(data_dir, 'VI_preidx_attn_val_lr1e3_hidden512_bleu_TF'))

RuntimeError: CuDNN error: CUDNN_STATUS_EXECUTION_FAILED